Preprocessing ideas: 
1) Removal of not arabic 
2) Removal and saving of diacritics 
3) Segementation
4) Tokenization
5) Lemmatization

Feature Extraction ideas: 
1) POS
2) Morphological quadruples
3) Context
4) Last character


In [1]:
!pip install pyarabic

In [2]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

c:\Users\Mai\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import re
from Input_Utils import input_utils
from Preprocessing import preprocessing_utils
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, is_arabicrange, strip_diacritics


In [4]:
def remove_diacritics_and_save(text):
    stripped = araby.strip_diacritics(text)
    diacritics_dic = [{i: char} for i,char in enumerate(text) if char not in stripped]
    return stripped, diacritics_dic

In [5]:
def remove_non_arabic_char(text):
    regex = re.compile('[^؀-ۿ ]')
    result = regex.sub('', text)
    return result

In [6]:
train_dataset = input_utils.read_input('Dataset/train.txt')
text = train_dataset[0]
result = remove_non_arabic_char(text)
stripped, diacritics = remove_diacritics_and_save(result)

In [27]:
longest_sentence = ""
shortest_sentence = ""
max_len = 0
min_len = 1000000000
for t in train_dataset:
    t,_ = remove_diacritics_and_save(t)
    if (len(t) > max_len):
        max_len = len(t)
        longest_sentence= t
        
    if (len(t) < min_len):
        min_len = len(t)
        shortest_sentence = t
        

print(f"longest sentence is {longest_sentence} with length = {max_len}")
print(f"shortest sentence is {shortest_sentence} with length = {min_len}")

longest sentence is والله أسأل ، وبرسوله أتوسل ، أن ينفع به كما نفع بأصله ، وأن يغفر لمن نظر فيه بعين الإنصاف ، ودعا لمؤلفه بأن يدركه ربه جل وعلا بخفي الألطاف ، وبأن يمتعه بالنظر إلى وجهه ، ويمده بالإسعاف ، وحسبنا الله ونعم الوكيل ، ولا حول ولا قوة( 29 / 477 ). قوله : ( لاعن امرأته ) قال في الفتح : اللعان مأخوذ من اللعن ; لأن الملاعن يقول في الخامسة : لعنة الله عليه إن كان من الكاذبين , واختير لفظ اللعن دون الغضب في التسمية لأنه قول الرجل وهو الذي بدئ به في الآية , وهو أيضا يبدأ به . وقيل : سمي لعانا لأن اللعن : الطرد والإبعاد , وهو مشترك بينهما . وإنما خصت المرأة بلفظ الغضب لعظم الذنب بالنسبة إليها . ثم قال : وأجمعوا على أن اللعان مشروع , وعلى أنه لا يجوز مع عدم التحقق . واختلف في وجوبه على الزوج . وظاهر أحاديث الباب أن اللعان إنما يشرع بين الزوجين , وكذلك قوله تعالى : { والذين يرمون أزواجهم } الآية , فلو قال أجنبي لأجنبية : يا زانية وجب عليه حد القذف . قوله : ( ففرق رسول الله صلى الله عليه وسلم بينهما ) استدل به من قال إن الفرقة بين المتلاعنين لا تقع بنفس اللعان حتى يوقعها الحاكم وأج

1885.75


In [7]:
!pip install qalsadi
import qalsadi

In [8]:
import qalsadi.lemmatizer

In [9]:
lemmer = qalsadi.lemmatizer.Lemmatizer()
lemmas = lemmer.lemmatize_text('فعل')
lemmas

['عل']

In [10]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ --no-deps FarasaPy3

Looking in indexes: https://test.pypi.org/simple/


In [11]:
%pip install FarasaPy3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement FarasaPy3 (from versions: none)
ERROR: No matching distribution found for FarasaPy3


In [12]:
import FarasaPy3
from FarasaPy3.api import FarasaPy3
farasaApi = FarasaPy3()
result = farasaApi.Segmentation(text)
print(result)

ModuleNotFoundError: No module named 'FarasaPy3'

#Features:
1) last char identity
2) raw word
3) context
4) context class
5) POS

In [13]:
!pip install farasa

In [16]:
import farasa
# import FarasaPOSTagger
farasa_pos_tagger = farasa.farasa_pos_tagger(interactive=True)

text = "هذا مثال على استخدام تسميات أجزاء الكلام في اللغة العربية."
pos_tags = farasa_pos_tagger.tag(text)
print(pos_tags)

AttributeError: module 'farasa' has no attribute 'farasa_pos_tagger'

In [17]:
!pip install arabicnlp

^C


In [24]:
sentence = "يذهب علي"
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)
print(tagged)

[('يذهب', 'NN'), ('علي', 'NN')]


  Using cached arabicnlp-0.1.7-py3-none-any.whl (14.4 MB)
  Using cached keras-3.0.1-py3-none-any.whl (999 kB)
  Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached dm_tree-0.1.8-cp39-cp39-win_amd64.whl (101 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached rich-13.7.0-py3-none-any.whl (240 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Mai\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Mai\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\Mai\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\Mai\anaconda3\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\Mai\anaconda3\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\Mai\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Mai\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\Mai\anaconda3\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
ConnectionResetErro

In [ ]:
import arabicnlp

ModuleNotFoundError: No module named 'arabicnlp'

In [42]:
# import json
# import requests
# url = 'https://farasa.qcri.org/webapi/pos/'
# text = 'يُشار إلى أن اللغة العربية' 
# api_key = "#####################"
# payload = {'text': text, 'api_key': api_key}
# data = requests.post(url, data=payload)
# result = json.loads(data.text)
# print(result) 

In [43]:
%pip install stanfordnlp

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import stanfordnlp

# Set the path to the Arabic POS Tagger JAR file and models
arabic_pos_tagger_path = '/path/to/stanford-postagger-3.9.2.jar'
arabic_pos_model_path = '/path/to/arabic-ud.tagger'

# Initialize the Arabic POS Tagger
arabic_pos_tagger = stanfordnlp.Pipeline(processors='tokenize,pos', pos_model_path=arabic_pos_model_path, pos_jar_path=arabic_pos_tagger_path, lang='ar')

# Example text
text = "هذا مثال على استخدام تسميات أجزاء الكلام في اللغة العربية."

# Process the text and obtain POS tags
doc = arabic_pos_tagger(text)

# Access POS tags for each word in the text
for sentence in doc.sentences:
    for word in sentence.words:
        print(f'{word.text}: {word.pos}')


ModuleNotFoundError: No module named 'stanfordnlp'